In [1]:
import os
import omegaconf
from shell.utils.experiment_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd

In [2]:
def get_cfg(save_root_dir = "experiment_results/toy_fedprox",
    dataset = "mnist",
    algo = "monolithic",
    num_train = 64,
    seed = 0,
    use_contrastive = True,):
    job_name = f"{dataset}_{algo}_numtrain_{num_train}"
    if use_contrastive:
        job_name += "_contrastive"
    experiment = os.path.join(save_root_dir, job_name, dataset,algo, f"seed_{seed}")
    config_path = os.path.join(experiment, "hydra_out", ".hydra", "config.yaml")
    # read the config file
    cfg = omegaconf.OmegaConf.load(config_path)
    graph, datasets, NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg,fleet_additional_cfg = setup_experiment(cfg)
    net_cfg['num_tasks'] = 2
    return graph, datasets, NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg,fleet_additional_cfg, cfg


In [3]:
graph, datasets, NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg,fleet_additional_cfg, cfg = get_cfg("experiment_results/toy_fedprox")

{'train': {'component_update_freq': 10, 'num_epochs': 10, 'init_component_update_freq': 10, 'init_num_epochs': 10, 'save_freq': 1}, 'dataset': {'dataset_name': 'mnist', 'num_tasks': 6, 'num_classes_per_task': 2, 'with_replacement': True, 'num_trains_per_class': 64, 'num_vals_per_class': 50, 'remap_labels': True}, 'net': {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.0}, 'sharing_strategy': {'name': 'fedprox', 'num_coms_per_round': 1, 'comm_freq': 1, 'mu': 0.0}, 'seed': 0, 'algo': 'monolithic', 'job_name': 'mnist_monolithic_numtrain_64_contrastive', 'num_agents': 2, 'root_save_dir': 'experiment_results/toy_fedprox/', 'parallel': False, 'num_init_tasks': 4, 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${algo}/seed_${seed}', 'batch_size': 64, 'memory_size': 32, 'improvement_threshold': 0.05, 'use_contrastive': True}}
i_size 28
num_classes 2
net_cfg {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.0, 'i_size': 28, 'num_classes': 2, 'num_

In [4]:
task_id = 1
num_added_components = None
agent_id = 0
net1 = load_net(cfg, NetCls, net_cfg, agent_id=agent_id, task_id=task_id, num_added_components=num_added_components)

save_dir experiment_results/toy_fedprox//mnist_monolithic_numtrain_64_contrastive/mnist/monolithic/seed_0/agent_0/task_1


In [5]:
graph, datasets, NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg,fleet_additional_cfg, cfg = get_cfg("experiment_results/toy_fedavg")

{'train': {'component_update_freq': 10, 'num_epochs': 10, 'init_component_update_freq': 10, 'init_num_epochs': 10, 'save_freq': 1}, 'dataset': {'dataset_name': 'mnist', 'num_tasks': 6, 'num_classes_per_task': 2, 'with_replacement': True, 'num_trains_per_class': 64, 'num_vals_per_class': 50, 'remap_labels': True}, 'net': {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.0}, 'sharing_strategy': {'name': 'gradient', 'num_coms_per_round': 1, 'comm_freq': 1, 'mu': 1.0}, 'seed': 0, 'algo': 'monolithic', 'job_name': 'mnist_monolithic_numtrain_64_contrastive', 'num_agents': 2, 'root_save_dir': 'experiment_results/toy_fedavg/', 'parallel': False, 'num_init_tasks': 4, 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${algo}/seed_${seed}', 'batch_size': 64, 'memory_size': 32, 'improvement_threshold': 0.05, 'use_contrastive': True}}
i_size 28
num_classes 2
net_cfg {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.0, 'i_size': 28, 'num_classes': 2, 'num_

In [6]:
avg_net1 = load_net(cfg, NetCls, net_cfg, agent_id=agent_id, task_id=task_id, num_added_components=num_added_components)

save_dir experiment_results/toy_fedavg//mnist_monolithic_numtrain_64_contrastive/mnist/monolithic/seed_0/agent_0/task_1


In [7]:
def compare(linear1, linear2):
    w_same = torch.allclose(linear1.weight, linear2.weight)
    b_same = torch.allclose(linear1.bias, linear2.bias)
    return w_same, b_same

In [8]:
compare(avg_net1.random_linear_projection, net1.random_linear_projection)

(True, True)

In [9]:
compare(avg_net1.decoder[1], net1.decoder[1])

(False, False)

In [10]:
compare(avg_net1.components[0], net1.components[0])

(True, True)

In [11]:
compare(avg_net1.components[1], net1.components[1])

(True, True)

In [12]:
compare(avg_net1.components[2], net1.components[2])

(True, True)

In [13]:
# compare(avg_net1.decoder[3], net1.decoder[3])

In [14]:
dataset = datasets[0]
task_id = 1
testloaders = {task: torch.utils.data.DataLoader(testset,
                                                         batch_size=256,
                                                         shuffle=False,
                                                         num_workers=4,
                                                         pin_memory=True,
                                                         ) for task, testset in enumerate(dataset.testset[:(task_id+1)])}

In [15]:
eval_net(net1, testloaders)

{0: 0.5592948717948718, 1: 0.6556224899598394, 'avg': 0.6074586808773557}

In [16]:
eval_net(avg_net1, testloaders)

{0: 0.5592948717948718, 1: 0.6019076305220884, 'avg': 0.5806012511584802}